In [2]:
import tensorflow.contrib.slim as slim
import tensorflow as tf

In [3]:
#  Model Variables can be saved in the model 
weights = slim.model_variable('weights',shape=[10,10,3,3],
                              initializer=tf.truncated_normal_initializer(stddev=0.1),
                              regularizer=slim.regularizers.l2_regularizer(0.05))

In [6]:
model_variable = slim.get_model_variables()

In [7]:
model_variable

[<tf.Variable 'weights:0' shape=(10, 10, 3, 3) dtype=float32_ref>]

In [8]:
reg_variable = slim.variable(name='reg',
                            shape=[10,10,3],
                            initializer = tf.zeros_initializer())

In [9]:
slim.get_local_variables()

[]

In [10]:
slim.get_variables()

[<tf.Variable 'weights:0' shape=(10, 10, 3, 3) dtype=float32_ref>,
 <tf.Variable 'reg:0' shape=(10, 10, 3) dtype=float32_ref>]

In [18]:
inputs=slim.variable(shape=[224,224,3], name='inputs')

ValueError: Variable inputs already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "C:\Users\Lei\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\tensorflow\python\framework\ops.py", line 1204, in __init__
    self._traceback = self._graph._extract_stack()  # pylint: disable=protected-access
  File "C:\Users\Lei\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\tensorflow\python\framework\ops.py", line 2630, in create_op
    original_op=self._default_original_op, op_def=op_def)
  File "C:\Users\Lei\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 767, in apply_op
    op_def=op_def)


In [17]:
slim.conv2d(inputs=inputs, kernel_size=[2,2], stride=1, scope='conv1_1', num_outputs=128, padding='SAME')
slim.max_pool2d(inputs=inputs, kernel_size=[2,2], stride=1, scope='pool1')

ValueError: ('Convolution not supported for input with rank', 2)

In [12]:
slim.repeat(inputs=inputs, repetitions=3, layer=slim.conv2d, num_outputs=128, kernel_size=[2,2], stride=1, scope='conv1')

NameError: name 'inputs' is not defined

In [13]:
slim.fully_connected(inputs=inputs, num_outputs=32, scope='fc/fc_1')

NameError: name 'inputs' is not defined

In [23]:
slim.stack(inputs=inputs, layer=slim.fully_connected, stack_args=[32,64,128])

<tf.Tensor 'Stack_1/fully_connected_3/Relu:0' shape=(12, 128) dtype=float32>

In [24]:
slim.stack(inputs=inputs, layer=slim.conv2d, stack_args=[(32,[3,3]),(32,[1,1])])

ValueError: ('Convolution not supported for input with rank', 2)

In [26]:
# set in arg_scope the same parameters
with slim.arg_scope([slim.conv2d], padding='SAME',  
                    weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
                    weights_regularizer=slim.l2_regularizer(0.0005)):  
    net = slim.conv2d(inputs, 64, [11, 11], scope='conv1')  
    net = slim.conv2d(net, 128, [11, 11], padding='VALID', scope='conv2')  
    net = slim.conv2d(net, 256, [11, 11], scope='conv3')  

ValueError: ('Convolution not supported for input with rank', 2)

In [28]:
with slim.arg_scope([slim.conv2d, slim.fully_connected],  
                      activation_fn=tf.nn.relu,  
                      weights_initializer=tf.truncated_normal_initializer(stddev=0.01),  
                      weights_regularizer=slim.l2_regularizer(0.0005)):  
    with slim.arg_scope([slim.conv2d], stride=1, padding='SAME'):  
        net = slim.conv2d(inputs=inputs, num_outputs=64, kernel_size=[11, 11], stride=4, padding='VALID', scope='conv1')  
        net = slim.conv2d(net, 256, [5, 5],  
                      weights_initializer=tf.truncated_normal_initializer(stddev=0.03),  
                      scope='conv2')  
        net = slim.fully_connected(net, 1000, activation_fn=None, scope='fc')  

ValueError: ('Convolution not supported for input with rank', 2)

In [29]:
slim.losses.softmax_cross_entropy()
slim.losses.get_total_loss()

Instructions for updating:
Use tf.losses.softmax_cross_entropy instead. Note that the order of the logits and labels arguments has been changed.


TypeError: softmax_cross_entropy() missing 2 required positional arguments: 'logits' and 'onehot_labels'

# save variable 

In [31]:
variables_to_restore=slim.get_variables_by_name(given_name='inputs')

In [ ]:
restorer = tf.train.Saver(variables_to_restore)  
  
with tf.Session() as sess:  
  # Restore variables from disk.  
    restorer.restore(sess, "/tmp/model.ckpt")  
    print("Model restored.")  

In [34]:
variables_to_restore[0].op.name

'inputs'

In [ ]:
def name_in_checkpoint(var):  
    return 'vgg16/' + var.op.name  
  
variables_to_restore = slim.get_model_variables()  
variables_to_restore = {name_in_checkpoint(var):var for var in variables_to_restore}  
restorer = tf.train.Saver(variables_to_restore)  
  
with tf.Session() as sess:  
    # Restore variables from disk.  
    restorer.restore(sess, "/tmp/model.ckpt")  

In [36]:
def vgg16(inputs):  
    with slim.arg_scope([slim.conv2d, slim.fully_connected],  
                      activation_fn=tf.nn.relu,  
                      weights_initializer=tf.truncated_normal_initializer(0.0, 0.01),  
                      weights_regularizer=slim.l2_regularizer(0.0005)):  
        net = slim.repeat(inputs, 2, slim.conv2d, 64, [3, 3], scope='conv1')  
        net = slim.max_pool2d(net, [2, 2], scope='pool1')  
        net = slim.repeat(net, 2, slim.conv2d, 128, [3, 3], scope='conv2')  
        net = slim.max_pool2d(net, [2, 2], scope='pool2')  
        net = slim.repeat(net, 3, slim.conv2d, 256, [3, 3], scope='conv3')  
        net = slim.max_pool2d(net, [2, 2], scope='pool3')  
        net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv4')  
        net = slim.max_pool2d(net, [2, 2], scope='pool4')  
        net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv5')  
        net = slim.max_pool2d(net, [2, 2], scope='pool5')  
        net = slim.fully_connected(net, 4096, scope='fc6')  
        net = slim.dropout(net, 0.5, scope='dropout6')  
        net = slim.fully_connected(net, 4096, scope='fc7')  
        net = slim.dropout(net, 0.5, scope='dropout7')  
        net = slim.fully_connected(net, 1000, activation_fn=None, scope='fc8')  
    return net  